In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tqdm
!pip install gdown
!pip install -U ckip-transformers

In [3]:
root = '/content/drive/MyDrive/Colab Notebooks/QABot';
data_path = root + '/wiki_2021_08_05_1215639.json'


In [4]:
import json
with open(data_path, 'r', encoding='UTF-8') as f:
	data = json.load(f)

In [5]:
print(data[0])
print(len(data))

{'id': 1, 'title': '福斯家庭電影頻道', 'articles': '福斯家庭電影頻道（）是一個播放電影的亞洲電視頻道，由福斯國際頻道持有與出品，STAR衛視發佈。在香港、澳門、臺灣、菲律賓、新加坡、馬來西亞等地區均有業務。福斯家庭電影頻道已簽訂電影合約的單位包括20世紀福斯、迪斯尼、米高梅 - 梅耶，Studio Cana、獅門娛樂、頂峯娛樂、溫斯坦公司等。該電影頻道面向老人和兒童，主要提供家庭電影。'}
1215639


In [6]:
import ckip_transformers
from ckip_transformers.nlp import CkipWordSegmenter

import json
import sqlite3
from tqdm import tqdm
import re
import math

In [7]:
ws_driver = CkipWordSegmenter(level=1, device=0)

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [8]:
batch_size = 100
chunk_size = 100
reg = "[\s\-，.。\:：！!；;\?？（）\(\)\"\'《》〈〉．～—─\=「」『』、”“·／\#\[\]\$,－]"
skip_chunk = 0    # 重新載入點，預設為 0

In [9]:
epochs = int(math.ceil(len(data) / batch_size))
for epoch in tqdm(range(epochs)):
    if epoch < skip_chunk*chunk_size:
        continue

    if epoch % chunk_size == 0:
        f = open('%s/ws-result-level1/output-%07d-%07d.txt' % 
                 (root, epoch*batch_size, min((epoch+chunk_size)*batch_size-1, len(data)-1)), 'w', encoding='UTF-8')

    input = []
    for i in range(epoch*batch_size, min((epoch+1)*batch_size, len(data))):
        input.append(re.sub(reg, " ", data[i]['articles'] + " " + data[i]['title']))

    text = ws_driver(input, show_progress=False, batch_size = batch_size)

    j = 0
    for i in range(epoch*batch_size, min((epoch+1)*batch_size, len(data))):
        write = str(data[i]['id']) + ' '
        for key in text[j]:
            key = key.strip()
            if key != '':
                write += key + ' '
        j += 1
        write += '\n'
        f.write(write)

    if epoch % chunk_size == chunk_size-1 or epoch == epochs-1:
        print("success! %d" % (epoch))
        f.close()

  1%|          | 100/12157 [02:52<4:17:41,  1.28s/it]

success! 99


  2%|▏         | 200/12157 [05:22<5:05:41,  1.53s/it]

success! 199


  2%|▏         | 300/12157 [07:47<4:40:42,  1.42s/it]

...

success! 11999


100%|█████████▉| 12100/12157 [5:25:33<01:34,  1.65s/it]

success! 12099


100%|██████████| 12157/12157 [5:27:00<00:00,  1.61s/it]

success! 12156
